In [36]:
import string

def z_array(s):
    """ Use Z algorithm (Gusfield theorem 1.4.1) to preprocess s """
    assert len(s) > 1
    z = [len(s)] + [0] * (len(s)-1)
    # Initial comparison of s[1:] with prefix
    for i in range(1, len(s)):
        if s[i] == s[i-1]:
            z[1] += 1
        else:
            break
    r, l = 0, 0
    if z[1] > 0:
        r, l = z[1], 1
    for k in range(2, len(s)):
        assert z[k] == 0
        if k > r:
            # Case 1
            for i in range(k, len(s)):
                if s[i] == s[i-k]:
                    z[k] += 1
                else:
                    break
            r, l = k + z[k] - 1, k
        else:
            # Case 2
            # Calculate length of beta
            nbeta = r - k + 1
            zkp = z[k - l]
            if nbeta > zkp:
                # Case 2a: Zkp wins
                z[k] = zkp
            else:
                # Case 2b: Compare characters just past r
                nmatch = 0
                for i in range(r+1, len(s)):
                    if s[i] == s[i - k]:
                        nmatch += 1
                    else:
                        break
                l, r = k, r + nmatch
                z[k] = r - k + 1
    return z


def n_array(s):
    """ Compile the N array (Gusfield theorem 2.2.2) from the Z array """
    return z_array(s[::-1])[::-1]


def big_l_prime_array(p, n):
    """ Compile L' array (Gusfield theorem 2.2.2) using p and N array.
        L'[i] = largest index j less than n such that N[j] = |P[i:]| """
    lp = [0] * len(p)
    for j in range(len(p)-1):
        i = len(p) - n[j]
        if i < len(p):
            lp[i] = j + 1
    return lp


def big_l_array(p, lp):
    """ Compile L array (Gusfield theorem 2.2.2) using p and L' array.
        L[i] = largest index j less than n such that N[j] >= |P[i:]| """
    l = [0] * len(p)
    l[1] = lp[1]
    for i in range(2, len(p)):
        l[i] = max(l[i-1], lp[i])
    return l


def small_l_prime_array(n):
    """ Compile lp' array (Gusfield theorem 2.2.4) using N array. """
    small_lp = [0] * len(n)
    for i in range(len(n)):
        if n[i] == i+1:  # prefix matching a suffix
            small_lp[len(n)-i-1] = i+1
    for i in range(len(n)-2, -1, -1):  # "smear" them out to the left
        if small_lp[i] == 0:
            small_lp[i] = small_lp[i+1]
    return small_lp


def good_suffix_table(p):
    """ Return tables needed to apply good suffix rule. """
    n = n_array(p)
    lp = big_l_prime_array(p, n)
    return lp, big_l_array(p, lp), small_l_prime_array(n)


def good_suffix_mismatch(i, big_l_prime, small_l_prime):
    """ Given a mismatch at offset i, and given L/L' and l' arrays,
        return amount to shift as determined by good suffix rule. """
    length = len(big_l_prime)
    assert i < length
    if i == length - 1:
        return 0
    i += 1  # i points to leftmost matching position of P
    if big_l_prime[i] > 0:
        return length - big_l_prime[i]
    return length - small_l_prime[i]


def good_suffix_match(small_l_prime):
    """ Given a full match of P to T, return amount to shift as
        determined by good suffix rule. """
    return len(small_l_prime) - small_l_prime[1]


def dense_bad_char_tab(p, amap):
    """ Given pattern string and list with ordered alphabet characters, create
        and return a dense bad character table.  Table is indexed by offset
        then by character. """
    tab = []
    nxt = [0] * len(amap)
    for i in range(0, len(p)):
        c = p[i]
        assert c in amap
        tab.append(nxt[:])
        nxt[amap[c]] = i+1
    return tab


class BoyerMoore(object):
    """ Encapsulates pattern and associated Boyer-Moore preprocessing. """
    
    def __init__(self, p, alphabet='ACGT'):
        self.p = p
        self.alphabet = alphabet
        # Create map from alphabet characters to integers
        self.amap = {}
        for i in range(len(self.alphabet)):
            self.amap[self.alphabet[i]] = i
        # Make bad character rule table
        self.bad_char = dense_bad_char_tab(p, self.amap)
        # Create good suffix rule table
        _, self.big_l, self.small_l_prime = good_suffix_table(p)
    
    def bad_character_rule(self, i, c):
        """ Return # skips given by bad character rule at offset i """
        assert c in self.amap
        ci = self.amap[c]
        assert i > (self.bad_char[i][ci]-1)
        return i - (self.bad_char[i][ci]-1)
    
    def good_suffix_rule(self, i):
        """ Given a mismatch at offset i, return amount to shift
            as determined by (weak) good suffix rule. """
        length = len(self.big_l)
        assert i < length
        if i == length - 1:
            return 0
        i += 1  # i points to leftmost matching position of P
        if self.big_l[i] > 0:
            return length - self.big_l[i]
        return length - self.small_l_prime[i]
    
    def match_skip(self):
        """ Return amount to shift in case where P matches T """
        return len(self.small_l_prime) - self.small_l_prime[1]

In [37]:
# GCTAGCTCTACGAGTCTA
p = 'TCAA'
p_bm = BoyerMoore(p, alphabet='ACGT')
p_bm.bad_character_rule(2, 'T')


2

In [38]:
# GCTAGCTCTACGAGTCTA
# ACTA
p = 'ACTA'
p_bm = BoyerMoore(p, alphabet='ACGT')
p_bm.good_suffix_rule(0)

3

In [39]:
# ACACGCTCTACGAGTCTA
# ACAC
p = 'ACAC'
p_bm = BoyerMoore(p, alphabet='ACGT')
p_bm.match_skip()

2

In [40]:
def boyer_moore(p, p_bm, t):
    """ Do Boyer-Moore matching """
    i = 0
    occurrences = []
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences

In [41]:
t = 'GCTAGCTCTACGAGTCTA'
p = 'TCTA'
p_bm = BoyerMoore(p, alphabet='ACGT')
boyer_moore(p, p_bm, t)

[6, 14]

In [66]:
def approximate_match(p, t, n):
    positions = ()
    segment_length = int(round(len(p) / (n+1)))
    all_matches = set()
    for i in range(n+1):
        start = i*segment_length
        end = min((i+1)*segment_length, len(p))
        p_bm = BoyerMoore(p[start:end], alphabet='ACGT')
        matches = boyer_moore(p[start:end], p_bm, t)
        # Extend matching segments to see if whole p matches
        for m in matches:
            if m < start or m-start+len(p) > len(t):
                continue
            mismatches = 0
            for j in range(0, start):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            for j in range(end, len(p)):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            if mismatches <= n:
                all_matches.add(m - start)
    return list(all_matches)
    positions.append(str(i))  # Store the starting position as a string

    positions.sort() # Sort the positions

    return " ".join(positions)  # Retur

In [43]:
from Bio import SeqIO

# Open the fasta file and read sequences
human_chrome = r"C:\Users\User\Downloads\chr1.GRCh38.excerpt.fasta"  # Replace with your file path

In [44]:
def readGenome(human_chrome):
    genome = ''
    with open(human_chrome, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

In [45]:
genome = readGenome(human_chrome)

In [72]:
def approximate_match_with_mismatches(p, t, max_mismatches):
    """
    Finds approximate matches of pattern p in text t with up to max_mismatches.

    Args:
        p: The pattern string.
        t: The text string.
        max_mismatches: The maximum number of mismatches allowed.

    Returns:
        A list of 0-based starting indices of approximate matches in t.
    """
    positions = set()
    for i in range(len(t) - len(p) + 1):
        substring = t[i:i + len(p)]
        mismatches = 0
        for j in range(len(p)):
            if p[j] != substring[j]:
                mismatches += 1
                if mismatches > max_mismatches:
                    break
        if mismatches <= max_mismatches:
            positions.add(i)
    return sorted(list(positions))

In [76]:
p = 'AAAAA'
t = 'AACAAGCTGATAAACATTTAAAGAG'
print(approximate_match_with_mismatches(p, t, 2))

[0, 1, 8, 9, 10, 11, 12, 17, 18, 19, 20]


In [47]:
print(t[786435:786442])

GAGTTGG


In [48]:
from Bio import SeqIO
vibrio_chol = r"C:\Users\User\Downloads\Vibrio_cholerae.txt"

In [49]:
def readGenome(vibrio_chol):
    cholerae = ''
    with open(vibrio_chol, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                cholerae += line.rstrip()
    return cholerae

In [50]:
cholerae = readGenome(vibrio_chol)

In [61]:
p = 'CTTGATCAT'
t = cholerae
print(approximate_match(p, t, 0))


[200160, 152354, 152411, 197028, 129189, 376771, 60039, 163207, 98409, 532935, 622755, 392723, 1065555, 600085, 357976, 152283]


In [56]:
def order_and_remove_commas(numbers):
    
    if not numbers:
        return ""

    num_list = numbers.split(',')  # Split the string into a list of strings
    try:
        num_list = [int(num) for num in num_list]  # Convert the strings to integers
    except ValueError:
        return ""  # Return empty string if the input contains non-numeric values

    num_list.sort()  # Sort the list of integers in ascending order
    return " ".join(map(str, num_list))  # Convert the integers

In [75]:
numbers = '0, 2, 10, 12, 13, 19, 25, 41, 42, 46, 49, 56, 57, 61, 62, 64, 67, 70, 79, 82, 89, 90, 94, 97, 102, 106, 114, 118, 119, 125, 134, 140, 142, 147, 153, 160, 161, 167, 179, 197, 206, 209, 216, 223, 240, 245, 252, 253, 259, 262, 267, 269, 274, 282, 283, 290, 296, 300, 303, 307, 308, 315, 317, 319, 320, 325, 332, 336, 339, 343, 346, 353, 363, 373, 386, 388, 392, 393, 395, 400, 401, 406, 413, 414, 418, 421, 422, 426, 431, 435, 437, 448, 457, 458, 463, 468, 471, 474, 478, 481, 482, 483, 488, 498, 504, 506, 510, 511, 514, 515, 517, 519, 522, 531, 550, 554, 556, 559, 567, 570, 574, 582, 587, 588, 589, 593, 595, 598, 604, 611, 621, 623, 624, 637, 648, 655, 656, 658, 662, 667, 668, 669, 673, 674, 675, 680, 683, 685, 692, 694, 699, 708, 720, 722, 726, 728, 734, 738, 741, 746, 749, 753, 759, 760, 762, 764, 771, 772, 775, 778, 782, 787, 791, 794, 795, 803, 807, 813, 818, 820, 821, 825, 827, 828, 832, 837, 842, 844, 845, 848, 851, 854, 858, 861, 868, 869, 870, 874, 875, 880, 881, 886, 896, 898, 899, 903, 904, 906, 911, 913, 917, 933, 938, 939, 944, 953, 959, 960, 966, 967, 972, 973, 978, 981, 987, 990, 994, 996, 1001, 1002, 1005, 1006, 1009, 1012, 1015, 1016, 1017, 1020, 1022, 1027, 1032, 1037, 1039, 1044, 1054, 1059, 1066, 1067, 1078, 1084, 1087, 1096, 1099, 1103, 1105, 1106, 1111, 1112, 1118, 1131, 1138, 1141, 1142, 1150, 1158, 1159, 1165, 1172, 1174, 1191, 1192, 1195, 1198, 1202, 1210, 1212, 1215, 1226, 1229, 1236, 1240, 1242, 1243, 1248, 1253, 1254, 1260, 1261, 1264, 1267, 1271, 1272, 1274, 1278, 1281, 1288, 1289, 1296, 1299, 1301, 1306, 1307, 1308, 1311, 1318, 1321, 1329, 1338, 1346, 1349, 1350, 1366, 1368, 1375, 1380, 1398, 1404, 1405, 1413, 1414, 1431, 1438, 1445, 1450, 1456, 1462, 1469, 1472, 1476, 1477, 1482, 1484, 1489, 1503, 1515, 1519, 1521, 1522, 1528, 1529, 1536, 1538, 1545, 1546, 1553, 1554, 1556, 1561, 1563, 1568, 1569, 1573, 1581, 1584, 1596, 1598, 1601, 1603, 1604, 1610, 1619, 1623, 1633, 1636, 1644, 1646, 1655, 1658, 1663, 1670, 1674, 1677, 1688, 1689, 1696, 1702, 1703, 1707, 1709, 1710, 1713, 1714, 1715, 1718, 1725, 1726, 1729, 1733, 1741, 1742, 1748, 1749, 1757, 1764, 1788, 1796, 1797, 1802, 1804, 1806, 1809, 1810, 1813, 1817, 1820, 1824, 1825, 1826, 1831, 1832, 1836, 1846, 1858, 1862, 1865, 1868, 1869, 1874, 1876, 1877, 1880, 1883, 1884, 1890, 1891, 1897, 1901, 1904, 1916, 1917, 1923, 1926, 1937, 1941, 1945, 1947, 1948, 1954, 1957, 1963, 1964, 1965, 1969, 1970, 1972, 1973, 1977, 1980, 1983, 1988, 1989, 1990, 1991, 1992, 1994, 1995, 1998, 1999, 2016, 2022, 2023, 2028, 2030, 2050, 2052, 2053, 2056, 2058, 2059, 2062, 2069, 2079, 2080, 2084, 2086, 2092, 2102, 2103, 2109, 2113, 2116, 2117, 2119, 2124, 2126, 2131, 2135, 2136, 2139, 2144, 2146, 2149, 2154, 2156, 2157, 2161, 2163, 2164, 2166, 2170, 2175, 2184, 2198, 2205, 2210, 2228, 2232, 2235, 2241, 2242, 2246, 2249, 2253, 2257, 2263, 2279, 2280, 2286, 2287, 2289, 2296, 2298, 2307, 2308, 2311, 2325, 2328, 2333, 2335, 2338, 2345, 2348, 2355, 2363, 2365, 2366, 2372, 2374, 2375, 2376, 2382, 2386, 2394, 2406, 2407, 2410, 2413, 2417, 2418, 2424, 2425, 2429, 2431, 2432, 2434, 2438, 2450, 2456, 2459, 2460, 2463, 2467, 2474, 2477, 2478, 2483, 2486, 2493, 2500, 2505, 2511, 2519, 2522, 2526, 2529, 2549, 2556, 2561, 2562, 2571, 2580, 2587, 2595, 2597, 2598, 2605, 2613, 2615, 2616, 2620, 2621, 2622, 2623, 2625, 2629, 2632, 2636, 2639, 2651, 2658, 2659, 2672, 2675, 2679, 2681, 2682, 2688, 2690, 2691, 2695, 2696, 2698, 2699, 2706, 2707, 2713, 2717, 2726, 2750, 2759, 2766, 2767, 2772, 2773, 2784, 2787, 2789, 2792, 2796, 2798, 2799, 2800, 2804, 2805, 2810, 2811, 2812, 2813, 2818, 2821, 2828, 2835, 2840, 2847, 2852, 2853, 2859, 2863, 2866, 2874, 2881, 2884, 2891, 2899, 2902, 2906, 2913, 2921, 2929, 2930, 2932, 2933, 2943, 2947, 2954, 2962, 2963, 2970, 2986, 2988, 2993, 2994, 3001, 3004, 3011, 3018, 3020, 3026, 3027, 3036, 3043, 3046, 3049, 3050, 3056, 3057, 3058, 3059, 3062, 3064, 3066, 3069, 3071, 3078, 3079, 3082, 3083, 3088, 3089, 3095, 3096, 3101, 3103, 3104, 3113, 3114, 3126, 3130, 3132, 3133, 3136, 3139, 3150, 3154, 3162, 3164, 3173, 3176, 3181, 3197, 3201, 3208, 3212, 3215, 3221, 3226, 3228, 3234, 3235, 3242, 3246, 3254, 3256, 3261, 3285, 3288, 3293, 3294, 3301, 3305, 3308, 3313, 3315, 3318, 3325, 3328, 3329, 3336, 3339, 3351, 3352, 3360, 3372, 3377, 3378, 3379, 3382, 3388, 3390, 3395, 3402, 3403, 3410, 3411, 3414, 3419, 3422, 3425, 3433, 3444, 3448, 3449, 3454, 3456, 3464, 3468, 3469, 3470, 3471, 3472, 3473, 3475, 3478, 3479, 3482, 3489, 3490, 3496, 3497, 3512, 3513, 3515, 3519, 3522, 3527, 3533, 3542, 3549, 3552, 3559, 3560, 3567, 3571, 3576, 3579, 3582, 3588, 3591, 3593, 3604, 3609, 3610, 3617, 3620, 3624, 3631, 3633, 3634, 3638, 3641, 3651, 3663, 3685, 3686, 3697, 3703, 3706, 3713, 3715, 3720, 3726, 3730, 3731, 3735, 3738, 3751, 3770, 3771, 3774, 3776, 3778, 3782, 3783, 3787, 3790, 3791, 3793, 3796, 3797, 3802, 3803, 3807, 3808, 3811, 3814, 3818, 3819, 3820, 3821, 3826, 3832, 3833, 3841, 3842, 3848, 3849, 3852, 3865, 3872, 3876, 3877, 3880, 3894, 3896, 3900, 3911, 3914, 3917, 3921, 3924, 3929, 3932, 3936, 3943, 3955, 3958, 3960, 3963, 3965, 3977, 3981, 3990, 3993, 3996, 4004, 4011, 4014, 4026, 4041, 4046, 4053, 4057, 4060, 4067, 4073, 4074, 4075, 4081, 4082, 4088, 4109, 4118, 4121, 4126, 4128, 4129, 4130, 4134, 4136, 4138, 4141, 4145, 4148, 4152, 4153, 4155, 4163, 4169, 4185, 4186, 4190, 4196, 4206, 4208, 4215, 4223, 4229, 4233, 4236, 4245, 4246, 4251, 4252, 4254, 4261, 4264, 4273, 4276, 4278, 4279, 4280, 4282, 4290, 4291, 4296, 4299, 4304, 4307, 4309, 4312, 4318, 4320, 4323, 4326, 4327, 4334, 4338, 4346, 4361, 4363, 4373, 4376, 4377, 4394, 4406, 4413, 4415, 4419, 4424, 4427, 4444, 4448, 4452, 4461, 4465, 4470, 4478, 4479, 4482, 4485, 4486, 4491, 4492, 4502, 4503, 4509, 4510, 4519, 4520, 4526, 4535, 4540, 4571, 4576, 4578, 4581, 4583, 4586, 4589, 4596, 4602, 4603, 4605, 4608, 4623, 4626, 4633, 4643, 4650, 4654, 4659, 4661, 4667, 4668, 4679, 4691, 4695, 4696, 4698, 4702, 4703, 4709, 4710, 4715, 4717, 4722, 4738, 4750, 4754, 4758, 4759, 4776, 4777, 4790, 4796, 4803, 4804, 4806, 4807, 4812, 4813, 4819, 4820, 4829, 4847, 4854, 4858, 4861, 4867, 4872, 4875, 4876, 4882, 4884, 4886, 4890, 4891, 4892, 4898, 4905, 4910, 4915, 4916, 4917, 4921, 4923, 4931, 4936, 4944, 4951, 4955, 4959, 4960, 4963, 4970, 4975, 4976, 4983, 4984, 4991, 4993, 4999, 5000, 5001, 5006, 5009, 5013, 5017, 5020, 5024, 5025, 5032, 5039, 5041, 5051, 5054, 5057, 5059, 5066, 5071, 5074, 5078, 5080, 5081, 5088, 5094, 5095, 5098, 5100, 5104, 5106, 5109, 5111, 5114, 5125, 5134, 5136, 5139, 5147, 5156, 5157, 5160, 5177, 5178, 5185, 5187, 5201, 5202, 5208, 5209, 5211, 5212, 5219, 5231, 5234, 5239, 5242, 5244, 5249, 5255, 5257, 5267, 5277, 5283, 5288, 5294, 5295, 5301, 5308, 5310, 5311, 5316, 5318, 5325, 5331, 5332, 5338, 5343, 5353, 5356, 5368, 5379, 5390, 5395, 5397, 5407, 5411, 5414, 5418, 5421, 5424, 5430, 5433, 5435, 5440, 5445, 5451, 5454, 5456, 5461, 5483, 5485, 5492, 5493, 5500, 5503, 5515, 5517, 5522, 5524, 5529, 5532, 5539, 5543, 5562, 5566, 5569, 5581, 5583, 5592, 5593, 5598, 5599, 5600, 5605, 5618, 5623, 5629, 5634, 5636, 5641, 5643, 5646, 5652, 5655, 5659, 5661, 5664, 5666, 5667, 5676, 5679, 5680, 5685, 5687, 5692, 5697, 5700, 5704, 5707, 5717, 5723, 5724, 5726, 5729, 5730, 5732, 5736, 5741, 5753, 5759, 5769, 5772, 5773, 5779, 5789, 5793, 5795, 5796, 5802, 5803, 5808, 5817, 5823, 5824, 5826, 5831, 5836, 5840, 5841, 5848, 5849, 5853, 5858, 5862, 5875, 5876, 5882, 5893, 5896, 5897, 5900, 5903, 5904, 5906, 5911, 5916, 5919, 5920, 5926, 5928, 5930, 5931, 5933, 5936, 5937, 5938, 5943, 5945, 5946, 5950, 5953, 5964, 5967, 5976, 5979, 5983, 5992, 5994, 5999, 6001, 6002, 6008, 6012, 6013, 6019, 6034, 6036, 6041, 6044, 6055, 6061, 6062, 6069, 6072, 6074, 6079, 6087, 6088, 6093, 6095, 6101, 6103, 6110, 6114, 6117, 6119, 6122, 6128, 6135, 6152, 6153, 6154, 6162, 6175, 6179, 6182, 6184, 6190, 6197, 6204, 6205, 6209, 6213, 6217, 6220, 6227, 6238, 6248, 6251, 6259, 6261, 6268, 6270, 6277, 6301, 6306, 6307, 6311, 6313, 6314, 6317, 6319, 6320, 6325, 6336, 6339, 6345, 6346, 6352, 6353, 6355, 6360, 6361, 6366, 6371, 6378, 6381, 6388, 6393, 6396, 6399, 6405, 6406, 6407, 6416, 6417, 6425, 6427, 6428, 6435, 6436, 6439, 6444, 6446, 6460, 6466, 6467, 6468, 6480, 6506, 6512, 6513, 6516, 6520, 6522, 6524, 6534, 6539, 6545, 6547, 6552, 6557, 6558, 6559, 6565, 6568, 6578, 6582, 6589, 6590, 6592, 6602, 6605, 6610, 6612, 6614, 6619, 6626, 6627, 6628, 6634, 6635, 6643, 6649, 6652, 6654, 6657, 6659, 6662, 6665, 6668, 6672, 6679, 6682, 6684, 6690, 6692, 6693, 6695, 6700, 6703, 6705, 6712, 6713, 6715, 6716, 6720, 6723, 6731, 6737, 6746, 6756, 6762, 6763, 6769, 6773, 6775, 6777, 6780, 6791, 6793, 6799, 6800, 6816, 6817, 6822, 6825, 6829, 6836, 6840, 6850, 6859, 6866, 6870, 6873, 6877, 6880, 6888, 6895, 6897, 6898, 6904, 6905, 6914, 6917, 6924, 6927, 6935, 6938, 6947, 6948, 6951, 6953, 6954, 6957, 6958, 6959, 6960, 6961, 6964, 6968, 6975, 6978, 6989, 6994, 6996, 7004, 7029, 7031, 7032, 7039, 7043, 7046, 7050, 7054, 7056, 7057, 7064, 7067, 7074, 7090, 7092, 7101, 7104, 7110, 7116, 7117, 7121, 7123, 7130, 7138, 7142, 7150, 7155, 7156, 7161, 7164, 7169, 7188, 7200, 7203, 7205, 7209, 7212, 7214, 7216, 7217, 7220, 7224, 7226, 7233, 7236, 7243, 7244, 7250, 7253, 7259, 7265, 7266, 7270, 7275, 7277, 7278, 7282, 7284, 7285, 7294, 7298, 7301, 7302, 7305, 7306, 7308, 7309, 7313, 7315, 7316, 7318, 7320, 7322, 7329, 7334, 7335, 7339, 7350, 7357, 7364, 7370, 7373, 7374, 7375, 7381, 7382, 7385, 7391, 7394, 7398, 7399, 7406, 7412, 7419, 7421, 7425, 7426, 7430, 7431, 7434, 7459, 7462, 7474, 7485, 7488, 7497, 7499, 7516, 7520, 7525, 7528, 7530, 7537, 7540, 7544, 7548, 7549, 7555, 7556, 7559, 7565, 7568, 7572, 7578, 7582, 7585, 7589, 7592, 7594, 7600, 7601, 7604, 7607, 7612, 7614, 7621, 7632, 7639, 7640, 7645, 7652, 7653, 7659, 7660, 7666, 7669, 7674, 7685, 7694, 7699, 7702, 7706, 7707, 7710, 7711, 7717, 7718, 7727, 7729, 7731, 7734, 7735, 7736, 7741, 7743, 7744, 7746, 7750, 7752, 7753, 7757, 7762, 7769, 7774, 7775, 7777, 7779, 7782, 7784, 7790, 7792, 7795, 7799, 7812, 7817, 7818, 7821, 7823, 7827, 7828, 7829, 7830, 7833, 7837, 7846, 7849, 7850, 7856, 7862, 7863, 7873, 7878, 7881, 7883, 7889, 7890, 7891, 7895, 7903, 7904, 7922, 7926, 7927, 7932, 7933, 7936, 7943, 7949, 7952, 7956, 7958, 7965, 7966, 7969, 7987, 7991, 7996, 7998, 7999, 8005, 8006, 8012, 8013, 8020, 8028, 8030, 8033, 8039, 8040, 8041, 8042, 8047, 8048, 8049, 8054, 8055, 8056, 8065, 8066, 8069, 8072, 8077, 8085, 8094, 8099, 8115, 8121, 8122, 8123, 8127, 8131, 8134, 8138, 8145, 8152, 8156, 8158, 8161, 8163, 8170, 8174, 8178, 8185, 8186, 8193, 8196, 8198, 8199, 8205, 8206, 8207, 8218, 8219, 8222, 8225, 8226, 8238, 8244, 8252, 8265, 8276, 8283, 8287, 8289, 8299, 8302, 8304, 8310, 8313, 8322, 8327, 8334, 8338, 8341, 8345, 8348, 8351, 8358, 8366, 8369, 8374, 8387, 8396, 8397, 8398, 8404, 8405, 8407, 8408, 8414, 8415, 8424, 8426, 8435, 8441, 8444, 8450, 8451, 8453, 8456, 8463, 8464, 8478, 8484, 8485, 8494, 8496, 8499, 8502, 8506, 8507, 8509, 8513, 8520, 8528, 8533, 8538, 8542, 8546, 8549, 8559, 8566, 8567, 8573, 8574, 8577, 8584, 8590, 8592, 8597, 8602, 8607, 8609, 8616, 8625, 8632, 8641, 8650, 8653, 8654, 8661, 8662, 8664, 8669, 8670, 8672, 8677, 8684, 8688, 8690, 8693, 8695, 8696, 8697, 8701, 8703, 8704, 8712, 8713, 8717, 8724, 8725, 8729, 8737, 8739, 8743, 8745, 8750, 8755, 8758, 8761, 8762, 8765, 8768, 8784, 8791, 8795, 8796, 8798, 8803, 8810, 8813, 8816, 8821, 8823, 8824, 8826, 8833, 8835, 8845, 8852, 8866, 8872, 8874, 8875, 8878, 8891, 8893, 8894, 8898, 8901, 8904, 8911, 8912, 8919, 8926, 8927, 8930, 8938, 8941, 8945, 8955, 8956, 8963, 8971, 8972, 8976, 8988, 8989, 8997, 9016, 9025, 9026, 9029, 9032, 9037, 9040, 9048, 9051, 9055, 9056, 9058, 9062, 9065, 9067, 9068, 9071, 9075, 9086, 9091, 9094, 9103, 9106, 9109, 9113, 9114, 9121, 9133, 9140, 9146, 9149, 9150, 9158, 9164, 9166, 9170, 9173, 9175, 9177, 9179, 9180, 9182, 9184, 9185, 9186, 9193, 9194, 9201, 9204, 9208, 9209, 9212, 9228, 9235, 9244, 9260, 9264, 9273, 9280, 9285, 9286, 9287, 9293, 9298, 9305, 9313, 9314, 9316, 9320, 9321, 9323, 9330, 9361, 9364, 9366, 9367, 9371, 9373, 9374, 9379, 9384, 9386, 9387, 9388, 9394, 9397, 9400, 9405, 9408, 9416, 9417, 9445, 9447, 9458, 9459, 9481, 9486, 9489, 9490, 9493, 9520, 9522, 9527, 9530, 9535, 9538, 9540, 9546, 9549, 9556, 9562, 9565, 9571, 9574, 9579, 9580, 9583, 9587, 9592, 9593, 9595, 9597, 9600, 9603, 9604, 9607, 9609, 9610, 9616, 9617, 9621, 9626, 9634, 9641, 9645, 9646, 9649, 9656, 9660, 9663, 9671, 9675, 9679, 9680, 9686, 9691, 9693, 9696, 9697, 9700, 9702, 9708, 9709, 9715, 9716, 9725, 9728, 9731, 9736, 9737, 9739, 9743, 9752, 9754, 9758, 9761, 9771, 9778, 9786, 9791, 9792, 9799, 9800, 9802, 9811, 9812, 9820, 9827, 9832, 9834, 9839, 9857, 9864, 9866, 9867, 9874, 9876, 9888, 9902, 9905, 9908, 9911, 9919, 9921, 9922, 9929, 9935, 9937, 9938, 9940, 9946, 9947, 9957, 9962, 9964, 9966, 9967, 9972, 9975, 9988, 9991, 9996, 9998, 9999, 10004, 10005, 10011, 10012, 10032, 10036, 10042, 10043, 10046, 10053, 10056, 10058, 10061, 10065, 10069, 10072, 10075, 10081, 10091, 10093, 10095, 10098, 10100, 10102, 10103, 10105, 10110, 10114, 10121, 10128, 10129, 10133, 10136, 10145, 10150, 10152, 10157, 10164, 10175, 10176, 10182, 10183, 10186, 10190, 10193, 10200, 10201, 10206, 10209, 10216, 10220, 10223, 10225, 10226, 10227, 10232, 10234, 10236, 10237, 10239, 10249, 10254, 10259, 10262, 10265, 10269, 10271, 10275, 10284, 10286, 10290, 10293, 10297, 10300, 10305, 10307, 10314, 10324, 10331, 10339, 10341, 10343, 10349, 10350, 10354, 10357, 10359, 10360, 10364, 10367, 10370, 10371, 10372, 10377, 10381, 10388, 10390, 10396, 10405, 10407, 10410, 10413, 10416, 10424, 10425, 10429, 10432, 10434, 10435, 10437, 10442, 10449, 10457, 10458, 10462, 10464, 10465, 10469, 10471, 10475, 10480, 10485, 10488, 10492, 10493, 10508, 10525, 10530, 10532, 10535, 10536, 10538, 10540, 10543, 10544, 10550, 10558, 10565, 10568, 10575, 10581, 10584, 10589, 10597, 10602, 10605, 10609, 10610, 10613, 10618, 10621, 10622, 10627, 10640, 10643, 10647, 10652, 10659, 10660, 10668, 10675, 10676, 10679, 10680, 10683, 10684, 10688, 10691, 10695, 10697, 10699, 10704, 10709, 10713, 10721, 10732, 10733, 10734, 10742, 10745, 10749, 10753, 10758, 10761, 10767, 10768, 10771, 10775, 10777, 10778, 10781, 10784, 10788, 10791, 10793, 10794, 10800, 10803, 10811, 10818, 10823, 10825, 10827, 10830, 10837, 10838, 10841, 10845, 10848, 10857, 10858, 10864, 10865, 10876, 10877, 10883, 10884, 10893, 10895, 10896, 10898, 10903, 10905, 10906, 10910, 10911, 10913, 10916, 10919, 10928, 10932, 10937, 10940, 10945, 10948, 10955, 10963, 10970, 10984, 10989, 10991, 10997, 11000, 11007, 11019, 11022, 11025, 11031, 11032, 11036, 11041, 11057, 11062, 11067, 11069, 11076, 11084, 11086, 11093, 11094, 11100, 11101, 11107, 11114, 11120, 11127, 11134, 11135, 11143, 11151, 11152, 11156, 11159, 11162, 11166, 11168, 11175, 11182, 11185, 11186, 11190, 11196, 11197, 11204, 11205, 11209, 11212, 11213, 11216, 11218, 11223, 11224, 11225, 11229, 11231, 11235, 11238, 11250, 11263, 11266, 11267, 11270, 11272, 11273, 11277, 11280, 11287, 11288, 11294, 11296, 11297, 11301, 11306, 11310, 11313, 11317, 11352, 11354, 11355, 11360, 11361, 11364, 11366, 11372, 11388, 11396, 11408, 11415, 11419, 11426, 11429, 11432, 11436, 11437, 11441, 11446, 11448, 11451, 11455, 11458, 11462, 11463, 11466, 11474, 11481, 11482, 11486, 11488, 11489, 11493, 11496, 11497, 11498, 11505, 11508, 11511, 11513, 11515, 11522, 11528, 11533, 11542, 11544, 11550, 11551, 11556, 11566, 11568, 11582, 11587, 11596, 11601, 11602, 11609, 11613, 11615, 11616, 11626, 11634, 11636, 11641, 11644, 11647, 11653, 11664, 11673, 11677, 11682, 11683, 11684, 11691, 11696, 11698, 11699, 11707, 11710, 11719, 11722, 11723, 11728, 11729, 11730, 11736, 11741, 11749, 11752, 11760, 11765, 11770, 11773, 11780, 11797, 11799, 11807, 11808, 11815, 11818, 11819, 11823, 11824, 11826, 11827, 11836, 11852, 11860, 11862, 11865, 11870, 11872, 11873, 11876, 11880, 11884, 11886, 11890, 11893, 11897, 11898, 11901, 11903, 11904, 11907, 11909, 11914, 11918, 11919, 11921, 11931, 11938, 11941, 11951, 11959, 11964, 11966, 11969, 11971, 11974, 11977, 11988, 11994, 11997, 12004, 12011, 12015, 12024, 12025, 12031, 12034, 12037, 12043, 12045, 12054, 12061, 12063, 12067, 12072, 12079, 12083, 12086, 12088, 12091, 12092, 12098, 12100, 12105, 12112, 12122, 12130, 12133, 12138, 12145, 12146, 12156, 12165, 12168, 12192, 12197, 12199, 12203, 12210, 12215, 12217, 12223, 12224, 12226, 12231, 12237, 12238, 12240, 12243, 12244, 12250, 12253, 12255, 12260, 12262, 12267, 12270, 12277, 12283, 12284, 12287, 12294, 12295, 12305, 12310, 12313, 12317, 12320, 12328, 12329, 12332, 12334, 12336, 12340, 12343, 12348, 12349, 12354, 12355, 12356, 12362, 12365, 12367, 12370, 12377, 12390, 12405, 12412, 12419, 12424, 12425, 12429, 12432, 12435, 12436, 12441, 12442, 12446, 12458, 12466, 12472, 12475, 12481, 12482, 12483, 12484, 12489, 12490, 12491, 12495, 12496, 12497, 12499, 12500, 12503, 12505, 12506, 12512, 12525, 12528, 12531, 12543, 12549, 12550, 12556, 12560, 12569, 12573, 12575, 12578, 12579, 12581, 12582, 12589, 12590, 12591, 12599, 12602, 12604, 12612, 12614, 12621, 12623, 12624, 12629, 12630, 12632, 12633, 12635, 12640, 12644, 12649, 12653, 12659, 12660, 12661, 12662, 12664, 12667, 12672, 12679, 12683, 12690, 12693, 12698, 12700, 12701, 12707, 12708, 12709, 12715, 12726, 12729, 12733, 12734, 12740, 12748, 12750, 12751, 12755, 12758, 12767, 12773, 12778, 12785, 12789, 12802, 12804, 12805, 12806, 12812, 12814, 12822, 12824, 12825, 12829, 12832, 12834, 12836, 12837, 12841, 12843, 12844, 12851, 12855, 12857, 12860, 12873, 12889, 12897, 12900, 12903, 12907, 12910, 12915, 12916, 12918, 12919, 12921, 12929, 12930, 12939, 12946, 12952, 12954, 12959, 12961, 12966, 12977, 12980, 12989, 12995, 12996, 13002, 13007, 13008, 13009, 13014, 13016, 13019, 13021, 13022, 13027, 13028, 13032, 13034, 13035, 13036, 13039, 13045, 13046, 13050, 13053, 13058, 13073, 13078, 13083, 13091, 13100, 13102, 13105, 13109, 13111, 13112, 13115, 13116, 13118, 13123, 13125, 13127, 13131, 13141, 13147, 13149, 13155, 13160, 13164, 13167, 13171, 13173, 13178, 13180, 13181, 13187, 13188, 13195, 13196, 13203, 13204, 13211, 13212, 13218, 13224, 13227, 13228, 13234, 13235, 13241, 13242, 13243, 13244, 13249, 13253, 13256, 13268, 13270, 13275, 13277, 13281, 13289, 13293, 13295, 13299, 13301, 13305, 13306, 13309, 13312, 13317, 13323, 13324, 13336, 13342, 13343, 13344, 13347, 13353, 13354, 13360, 13371, 13372, 13376, 13379, 13384, 13385, 13393, 13394, 13401, 13402, 13405, 13409, 13412, 13413, 13414, 13419, 13420, 13426, 13427, 13447, 13450, 13458, 13461, 13467, 13468, 13469, 13472, 13475, 13482, 13486, 13487, 13494, 13509, 13513, 13516, 13526, 13533, 13541, 13544, 13546, 13563, 13564, 13567, 13569, 13575, 13576, 13579, 13582, 13588, 13593, 13603, 13606, 13617, 13618, 13621, 13624, 13628, 13631, 13632, 13638, 13644, 13656, 13660, 13661, 13663, 13668, 13687, 13688, 13694, 13695, 13701, 13702, 13709, 13716, 13717, 13718, 13723, 13728, 13731, 13738, 13746, 13750, 13759, 13760, 13768, 13781, 13785, 13791, 13792, 13793, 13794, 13801, 13808, 13812, 13813, 13815, 13817, 13819, 13820, 13828, 13829, 13835, 13838, 13839, 13845, 13852, 13857, 13865, 13881, 13882, 13883, 13887, 13893, 13896, 13898, 13904, 13909, 13912, 13926, 13932, 13936, 13942, 13944, 13951, 13955, 13962, 13966, 13980, 13985, 13987, 13988, 13996, 14000, 14003, 14005, 14006, 14009, 14012, 14013, 14016, 14019, 14022, 14039, 14040, 14044, 14049, 14054, 14056, 14061, 14063, 14064, 14067, 14070, 14073, 14080, 14097, 14108, 14112, 14115, 14123, 14130, 14148, 14149, 14154, 14155, 14161, 14162, 14168, 14169, 14172, 14176, 14177, 14179, 14201, 14207, 14213, 14214, 14234, 14236, 14240, 14259, 14262, 14263, 14264, 14267, 14270, 14271, 14277, 14278, 14286, 14292, 14295, 14297, 14300, 14303, 14310, 14315, 14316, 14319, 14321, 14323, 14326, 14328, 14329, 14335, 14336, 14342, 14344, 14351, 14358, 14367, 14373, 14374, 14380, 14381, 14382, 14396, 14405, 14410, 14413, 14415, 14421, 14422, 14426, 14430, 14433, 14440, 14443, 14448, 14455, 14459, 14463, 14464, 14466, 14470, 14480, 14487, 14490, 14494, 14496, 14497, 14503, 14505, 14506, 14509, 14511, 14514, 14523, 14534, 14536, 14543, 14545, 14550, 14554, 14555, 14563, 14565, 14581, 14588, 14591, 14594, 14596, 14598, 14602, 14608, 14614, 14620, 14626, 14633, 14637, 14656, 14662, 14666, 14670, 14677, 14680, 14684, 14687, 14693, 14701, 14702, 14706, 14707, 14708, 14713, 14714, 14723, 14724, 14729, 14731, 14736, 14737, 14743, 14756, 14759, 14761, 14764, 14767, 14768, 14775, 14782, 14783, 14787, 14788, 14795, 14796, 14801, 14803, 14806, 14809, 14814, 14817, 14824, 14825, 14827, 14837, 14838, 14845, 14848, 14852, 14855, 14861, 14868, 14871, 14873, 14882, 14886, 14889, 14892, 14896, 14897, 14902, 14903, 14909, 14912, 14931, 14937, 14943, 14947, 14949, 14954, 14959, 14962, 14967, 14974, 14981, 14988, 14989, 14992, 14995, 14996, 15000, 15002, 15010, 15023, 15030, 15034, 15037, 15038, 15046, 15047, 15049, 15053, 15055, 15060, 15061, 15066, 15068, 15071, 15074, 15078, 15085, 15086, 15092, 15093, 15094, 15099, 15107, 15113, 15114, 15116, 15124, 15129, 15136, 15140, 15148, 15149, 15152, 15156, 15157, 15162, 15165, 15171, 15176, 15178, 15191, 15198, 15209, 15214, 15225, 15228, 15234, 15235, 15241, 15248, 15253, 15255, 15265, 15289, 15290, 15297, 15300, 15302, 15303, 15312, 15318, 15319, 15322, 15325, 15334, 15339, 15342, 15346, 15349, 15351, 15353, 15354, 15358, 15371, 15387, 15393, 15394, 15398, 15399, 15400, 15406, 15411, 15413, 15419, 15420, 15422, 15426, 15428, 15429, 15431, 15435, 15446, 15461, 15465, 15471, 15472, 15476, 15479, 15491, 15492, 15496, 15498, 15499, 15505, 15513, 15514, 15521, 15526, 15533, 15540, 15545, 15547, 15548, 15554, 15555, 15557, 15558, 15561, 15563, 15575, 15579, 15586, 15587, 15592, 15595, 15599, 15604, 15606, 15607, 15613, 15616, 15619, 15620, 15629, 15636, 15639, 15642, 15650, 15652, 15653, 15654, 15659, 15660, 15662, 15666, 15677, 15683, 15690, 15691, 15694, 15701, 15707, 15708, 15715, 15721, 15722, 15727, 15728, 15732, 15740, 15745, 15748, 15751, 15755, 15756, 15758, 15761, 15762, 15764, 15771, 15775, 15777, 15784, 15795, 15800, 15802, 15803, 15814, 15818, 15829, 15834, 15836, 15841, 15842, 15845, 15850, 15859, 15866, 15873, 15879, 15885, 15891, 15900, 15901, 15906, 15912, 15919, 15922, 15928, 15932, 15933, 15940, 15941, 15949, 15951, 15956, 15963, 15967, 15980, 15983, 15984, 15987, 15989, 15990, 15991, 15996, 15997, 16008, 16010, 16015, 16018, 16023, 16030, 16040, 16042, 16048, 16051, 16052, 16057, 16058, 16060, 16065, 16068, 16077, 16089, 16092, 16103, 16108, 16111, 16115, 16116, 16122, 16123, 16127, 16135, 16136, 16138, 16141, 16143, 16144, 16146, 16147, 16149, 16150, 16151, 16158, 16159, 16165, 16166, 16167, 16172, 16183, 16190, 16193, 16195, 16211, 16217, 16218, 16221, 16224, 16228, 16231, 16234, 16239, 16248, 16253, 16255, 16259, 16261, 16269, 16270, 16276, 16289, 16294, 16297, 16308, 16311, 16313, 16318, 16325, 16329, 16333, 16335, 16341, 16342, 16347, 16351, 16353, 16354, 16360, 16365, 16371, 16372, 16378, 16381, 16386, 16389, 16393, 16397, 16400, 16404, 16408, 16411, 16412, 16417, 16424, 16428, 16431, 16435, 16438, 16445, 16447, 16457, 16467, 16474, 16478, 16483, 16498, 16512, 16517, 16524, 16525, 16540, 16543, 16551, 16552, 16568, 16575, 16577, 16578, 16585, 16592, 16595, 16598, 16602, 16605, 16606, 16609, 16611, 16617, 16619, 16644, 16660, 16667, 16677, 16684, 16686, 16687, 16692, 16693, 16703, 16708, 16710, 16712, 16719, 16723, 16728, 16729, 16734, 16735, 16737, 16738, 16745, 16746, 16755, 16756, 16758, 16760, 16767, 16784, 16786, 16787, 16793, 16794, 16799, 16800, 16802, 16804, 16807, 16808, 16811, 16815, 16817, 16824, 16827, 16843, 16852, 16874, 16885, 16895, 16903, 16907, 16910, 16920, 16923, 16928, 16929, 16931, 16937, 16938, 16941, 16944, 16951, 16957, 16964, 16970, 16973, 16977, 16980, 16984, 16991, 16998, 17003, 17006, 17012, 17020, 17023, 17032, 17036, 17038, 17039, 17045, 17054, 17057, 17058, 17064, 17066, 17071, 17088, 17095, 17102, 17103, 17123, 17130, 17136, 17142, 17143, 17149, 17150, 17151, 17157, 17159, 17164, 17169, 17170, 17188, 17190, 17191, 17195, 17202, 17212, 17219, 17221, 17222, 17224, 17226, 17228, 17242, 17245, 17247, 17250, 17258, 17262, 17277, 17282, 17286, 17289, 17291, 17293, 17301, 17308, 17314, 17315, 17317, 17321, 17322, 17324, 17327, 17330, 17335, 17339, 17340, 17343, 17347, 17355, 17358, 17362, 17365, 17368, 17371, 17373, 17376, 17381, 17383, 17387, 17388, 17389, 17390, 17392, 17396, 17397, 17401, 17404, 17405, 17411, 17415, 17421, 17422, 17427, 17428, 17429, 17433, 17442, 17445, 17448, 17449, 17451, 17452, 17459, 17475, 17479, 17482, 17485, 17490, 17491, 17498, 17508, 17511, 17515, 17518, 17520, 17522, 17527, 17539, 17546, 17548, 17556, 17560, 17563, 17566, 17573, 17577, 17579, 17580, 17584, 17585, 17587, 17594, 17596, 17599, 17601, 17606, 17608, 17613, 17617, 17624, 17629, 17630, 17634, 17635, 17637, 17639, 17650, 17654, 17657, 17660, 17664, 17666, 17671, 17674, 17690, 17705, 17709, 17716, 17727, 17731, 17735, 17738, 17742, 17743, 17751, 17752, 17764, 17773, 17775, 17781, 17794, 17795, 17801, 17802, 17811, 17818, 17819, 17824, 17830, 17835, 17837, 17839, 17846, 17853, 17860, 17871, 17876, 17879, 17883, 17892, 17898, 17902, 17907, 17910, 17913, 17918, 17921, 17932, 17939, 17942, 17943, 17949, 17950, 17953, 17954, 17960, 17967, 17974, 17975, 17978, 17987, 17993, 18000, 18003, 18007, 18010, 18011, 18013, 18014, 18015, 18016, 18023, 18031, 18044, 18050, 18055, 18057, 18059, 18073, 18080, 18081, 18083, 18087, 18088, 18092, 18100, 18105, 18111, 18118, 18122, 18123, 18125, 18130, 18131, 18134, 18135, 18141, 18142, 18148, 18155, 18158, 18163, 18164, 18165, 18166, 18169, 18170, 18174, 18176, 18181, 18182, 18185, 18188, 18190, 18195, 18200, 18207, 18211, 18229, 18232, 18233, 18241, 18244, 18246, 18248, 18249, 18251, 18255, 18262, 18268, 18269, 18275, 18276, 18282, 18283, 18285, 18292, 18294, 18298, 18316, 18322, 18325, 18328, 18329, 18336, 18341, 18344, 18348, 18350, 18351, 18356, 18376, 18382, 18383, 18385, 18394, 18399, 18400, 18401, 18404, 18406, 18407, 18409, 18411, 18414, 18416, 18421'
result = order_and_remove_commas(numbers)
print(f"Ordered numbers: {result}")


Ordered numbers: 0 2 10 12 13 19 25 41 42 46 49 56 57 61 62 64 67 70 79 82 89 90 94 97 102 106 114 118 119 125 134 140 142 147 153 160 161 167 179 197 206 209 216 223 240 245 252 253 259 262 267 269 274 282 283 290 296 300 303 307 308 315 317 319 320 325 332 336 339 343 346 353 363 373 386 388 392 393 395 400 401 406 413 414 418 421 422 426 431 435 437 448 457 458 463 468 471 474 478 481 482 483 488 498 504 506 510 511 514 515 517 519 522 531 550 554 556 559 567 570 574 582 587 588 589 593 595 598 604 611 621 623 624 637 648 655 656 658 662 667 668 669 673 674 675 680 683 685 692 694 699 708 720 722 726 728 734 738 741 746 749 753 759 760 762 764 771 772 775 778 782 787 791 794 795 803 807 813 818 820 821 825 827 828 832 837 842 844 845 848 851 854 858 861 868 869 870 874 875 880 881 886 896 898 899 903 904 906 911 913 917 933 938 939 944 953 959 960 966 967 972 973 978 981 987 990 994 996 1001 1002 1005 1006 1009 1012 1015 1016 1017 1020 1022 1027 1032 1037 1039 1044 1054 1059 1066 10

In [59]:
p = 'GCAAGCTGC'
t = 'GCGTGAGCAAGCTGGCAAGCTTGCAAGCTAGGCAAGCTGCAAGCTGCAAGCTATTAGCAAGCTAGCGAGAAAATGAAGTGCTCGTGGCAAGCTCAGCAAGCTACCCCGGCAAGCTGCCGGCAAGCTGCAAGCTAAGGGCAAGCTGCAAGCTTCGCAAGCTCCTCGTGCAAGCTATCGAGTGCAAGCTGCAAGCTTCCGCAAGCTACGCAAGCTCGGCAAGCTTGCAAGCTTTACGCAAGCTAGGCAAGCTGCAAGCTATGCAAGCTTTTTCATTTGCAAGCTGCAAGCTAGCAAGCTGCAAGCTGCAAGCTGCAAGCTGGCAAGCTCAGTGCAAGCTGCAAGCTTTAGCAAGCTGCAAGCTGCAAGCTGCAAGCTGCAAGCAAGCTAATTCTAAAAGCAAGCTAACATTTGCAAGCTGCAAGCTTCAGCAAGCTGCAAGCTTTGCAAGCTTGCGCAAGCTTTGCAAGCTTTGCTGCAAGCTATGCAAGCTAGGGAGCAAGCTTACTGCAAGCTACGCAAGCTGGGGCAAGCTGCAAGCTCTGACGGCAAGCTTGGTCACGCAAGCTCGCAAGCTCGAGCAAGCTGCAAGCTTGCAAGCTTATGCAAGCTTGGCAAGCTGCAAGCTGCAAGCTCAACGCAAGCTAGGGAAGCAAGCTAAAGCAAGCTGCAAGCTGAGGCCTGGGGGCAAGCTGATAGCAAGCTAGCAAGCTTTGGCAAGCTGGGGACACAAACCGGGGCAAGCTCTGCAAGCTATAGCAAGCTCCGGCAAGCTGCAAGCTAGCAAGCTTCACTCGCAAGCTGCAAGCTTAGCAAGCTAAGCAAGCTCGCAAGCTGCAAGCTGCAAGCTGATGCAAGCTCCAAGCAAGCTGGTGCAAGCTAGCAAGCTGGCAAGCTTGCAAGCTCCACGTGTCGCAAGCTCGGGGCAAGCTAACGCAAGCTTGTCGCAAGCTGCAAGCTTGAGGCGCTTGGCGCAAGCTGGCAAGCTGTGCAAGCTCCAGGTGGTTCTGCAAGCTGTCGCAAGCTCGGCAAGCTGCCCTACGAACGCAAGCTGACGGCGGTGCAAGCTAACATAAGCGCAAGCTTAGAATCGCAAGCTAGCAAGCTTGCAAGCTTGCAAGCTGCAAGCTGCAAGCTGCAAGCTGTGCAAGCTTCCGCAAGCTGGCAAGCTGCAAGCTATATGCAAGCTCGCAAGCTTGCAAGCTCCTGCAAGCTCTCGGGGCAAGCTCATACGCAAGCTGCAAGCTAGCAAGCTCTTGCAAGCTGAGCAAGCTGGCAAGCTGCAAGCTGCAAGCTTCGCAAGCTGCAAGCTCCGCAAGCTGCAAGCTATGAAGCTGAGTTGCAAGCTGCAAGCTGCAAGCTTGCAAGCTGCAAGCTCCTGCAAGCTCGGCAAGCTCCTTCCGCAAGCTAGCAAGCTCGCAAGCTTATGCAAGCTGCAAGCTCGTGGCAAGCTCTCATGCAAGCTCTTGGCAAGCTGCAAGCTGCAAGCTGCAAGCTAACGCAAGCTCGTACGCAAGCTTAGCAAGCTAGGGCAAGCTCGCAAGCTATGGCAAGCTAATGCAGGCAAGCTAGCAAGCTCGCAAGCTTGCAAGCTTGCAAGCTGCAAGCTACGGCAAGCTGAGGCAAGCTTGTGCAAGCTGCAAGCTGGGCAAGCTTAGCAAGCTGCGCAAGCTGCAAGCTGCAAGCTCCGGGGCAAGCTGCAAGCTCTTGTGCAAGCTGAGCAAGCTTGCAAGCTGTCCGCAAGCTCGCAAGCTTGTTGCAAGCTACCCTCAGTGGCAAGCTGCAAGCTTGCAAGCTAATCGAGCAAGCTATGCAAGCTGCAAGCTAGCAAGCTGCAAGCTATTTTGCAAGCTTTCGTATGCAAGCTGCAAGCTGCTAGCAAGCTAACCAGCAAGCTCAGCAAGCTGAGCAAGCTCGCAAGCTCGCAAGCTTGCAAGCTAAGAGACCGCAAGCTGCAAGCTAGATGGCGCAAGCTCCTGCAAGCTGCAAGCTGGCAAGCTAGCAAGCTAGGGCAAGCTGCAAGCTGCAAGCTATCGCAAGCTGCAAGCTACGAGCAAGCTGCAAGCTGCAAGCTATGCAAGCTGCAAGCTGCAAGCTTCTTGCGCAAGCTTGCAAGCTGCAAGCTGCAAGCTCATGCAAGCTATGCAAGCTTAGTTGCAAGCTAATAGGCAAGCTGCAAGCTCAGTCTGCATCGGGTGCAAGCTTTGCAAGCTTTAGCAAGCTGCAAGCTAAGGCTAGCAAGCTGGCAAGCTAATGCAAGCTTTGCAAGCTGTCGCAAGCTCGCAAGCTAACGCAAGCTGCAAGCTGGTCTTCGACCCAGGAACGGCAAGCTCGCAAGCTTGGCAAGCTGCAAGCTAACTTGCAAGCTTTTGCAAGCTGCAAGCTCGCAAGCTTGCAAGCTGGTGCAAGCTGCAAGCTGGGCAAGCTGGCCAGCAAGCTGCCCTTCACAGGTTATGCAAGCTGCAAGCTCGTTCGGAAGCAAGCTGGCCAGTTCTTAAGGGTTCCGCGCAAGCTCGAGTATGCAAGCTGCAAGCTGCAAGCTCGCGCGAACGCAAGCTGGCAAGCTAACGCAAGCTTGCAAGCTAGCAAGCTTAGCAAGCTTGCAAGCTGCAAGCTGGCAAGCTGGGAACCCTGGCAAGCTGCAGCTTAGCAAGCTGCAAGCTTGGCAAGCTAATCCGCAAGCTCGCAAGCTGCAAGCTGGGCAAGCTTGCAAGCTGCAGCAAGCTGGCAAGCTAGAGGCAAGCTAGCAAGCTGCAAGCTCGCAAGCTCCGCAAGCTGCTTGCAAGCTGCAAGCTGGTGCAGAGCAAGCTTCGTTTGCAAGCTAGTTTGTCAAACAAGCCAGATGGCAAGCTATGGCTATAGCAAGCTTAGAGCAAGCTGCAGACAGCAAGCTGGGCAAGCTCATTCCGCAAGCTATAATCGGCAAGCTGCAAGCTCCATGCAAGCTGCAAGCTTGACCTGCAAGCTACGCAAGCTGGCAAGCTGCAAGCTGGGTGCAAGCTCTGCGCGCACGCGCAAGCTGCAAGCTCCACTGCAAGCTACCGGCAAGCTTCTCGGCAAGCTATCTAGCAAGCTGGCGCCCAGCAAGCTGGCAAGCTAAGCAAGCTAGCAAGCTGCAAGCTCTGCAAGCTAGCAAGCTCTTGCAAGCTGAGCAAGCTAGCAAGCTACCGGCAAGCTATCAGCAAGCTCCGCAAGCTCACGGGGACGCAAGCTCGGGCAAGCTGCAAGCTAAATTGGGGCAAGCTCTAGCAAGCTCAGCAAGCTTGCAAGCTGCAAGCTTGCAAGCTGTATGCAAGCTTTGCCGAGCAAGCTCTCTGCAAGCTCTACTTGCAAGCTGCAAGCTGGCAAGCTAGGTGCAAGCTCGCAAGCTGTGAGCAGCAAGCTAGCAAGCTAAACCTGGCAAGCTGCCAGCGAGCAAGCTGCAAGCTGCACGCAAGCTGCAAGCTGCAAGCTGCAAGCTGACTTCTAAGCAAGCTGCAAGCTGCCTGCAAGCTAGGTGCAAGCTTCCAGTCACGCTGTACGCAAGCTGACTTTGCAAGCTGCAAGCTGCAAGCTGAGCAAGCTCATTGTGAGGCAAGCTAGTGCAAGCTGCAAGCTAGTGCAAGCTTCAGGCATACACCACCAGTGTGCAAGCTTCGCAAGCTACGCCAGGCAAGCTGGGCTTATTCACGTGCAAGCTGAGCAAGCTAGCAAGCTCTGTTAATGGCAAGCTGCAAGCTTCGCAAGCTTTGCAAGCTGCAAGCTGGCAAGCTAGCAAGCTGATGCAAGCTACGTAGTGCAAGCTCGATGCAAGCTGGCAAGCTGCGGCAAGCTAGCAAGCTGCAAGCTGCCTAGGCAAGCTGCAAGCTAAGTGTGGCAAGCTGCAAGCTGACGGTGCAAGCTGGCAAGCTTCGCAGCAAGCTACTATATATTAATCTCGTGCAAGCTGTGCAAGCTTTGCAAGCTGGGCAAGCTGCAAGCTAGGACTGGCAAGCTGCAAGCTTTGCAAGCTTGCAAGCTGCAAGCTGCAAGCTTAGCAAGCTGTGCAAGCTGACTTTATTCTGGAAGCAAGCTTCGCCCGCAAGCTGCAAGCTGGCGCCGAAGCAAGCTGGAGCAAGCTTGCAAGCTCGCAAGCTCGGCAAGCTGCTGGTGGCAAGCTGCAAGCTCGATGTAGCAAGCTGCAAGCTGCAAGCTCCAGAGGAGCAAGCTGCAAGCTGTGGTGGCAAGCTGCAAGCTCCTACAAAACTTCCTTCAGCAAGCTGAGCCGCAAGCTAGCAAGCTGTGCAAGCTGCAAGCTTTGCAAGCTCGTGGCAAGCTACTTCGCAAGCTCCGCAAGCTAGCAAGCTGCAAGCTATCCGCAAGCTGCAAGCTAACGCAAGCTTGCAAGCTTTACCCAGCAAGCTTCAGGTGGTTGCAAGCTTGGCAAGCTTGCAAGCTTGCAAGCTGCAAGCTGCAAGCTGCTTTTCTCCAGCAAGCTGAAGCAAGCTGTAGCAAGCTCGAGCAAGCTGCAAGCTCTTTAGGGGCAAGCTGGGCAAGCTCGTAGATTAAACCCCCCTGTATCCCAAACAGGCAAGCTGGCAAGCTCAGAGCAAGCTTGGCAAGCTTCAGCGAGCAAGCTGCAAGCTCTCGCAAGCTGGCAAGCTAGTCGCAAGCTAGCAAGCTTTCGGCAAGCTAGCAAGCTGAAGTAGCAAGCTCGGCAAGCTCTGCAAGCTGCAAGCTGCAAGCTTGCAAGCTTCTGCAAGCTGTAGCAAGCTCGCAAGCTCGCAAGCTGGGCAAGCTGCGCAAGCTGCAAGCTTCGCAAGCTTGCGCAAGCTGAGGGCAAGCTGCAAGCTTCGCAAGCTAAGCAAGCTGATGCCACTGGCAAGCTGTGCGCAAGCTGCGCAAGCTCAGCGATAGCAAGCTAGCAAGCTGCCTTCGCGCAAGCTCCCTTGCAAGCTGCAAGCTGCAAGCTGAGGCAAGCTGCAAGCTGCTCGGTTAGAGCAAGCTGCCCACGCAAGCTAGACGCAAGCTTGAGCAAGCTGCAAGCTAGCAAGCTGCAAGCTAGACCTGTGCAAGCTGCAAGCTGCAGCAAGCTGGCAAGCTGCAAGCTGCAAGCTGCAAGCTACGTTGCAAGCTTCATGGTAGCAAGCTGCAAGCTCACGCAAGCTGCAAGCTACTTTGTACCCGCAAGCTATCGGCAAGCTGCAAGCTGGGCAAGCTTCCGCAAGCTGCGCAAGCTTCGCAAGCTGCAAGCTTGCAAGCTGCAAGCTGATGCAAGCTCGCAAGCTTCACCTTGCAAGCTCTGCAAGCTGCAAGCTGGCAAGCTCTCCTGTAGCAAGCTCTGCAAGCTGCAAGCTGATGCGCAAGCTCTGCAAGCTCGCAAGCTTGCAAGCTGCAAGCTCACCGATGCAAGCTGGCAAGCTGGCAAGCTACGGCAAGCTGCAGGCAAGCTAAGCAAGCTTGTAGATAGGCAAGCTCGGGCAAGCTGCAAGCTAGCAAGCTAAGCAAGCTAGCAAGCTTCGAGCAAGCTGGCAAGCTTGTGCAAGCTTAACGCAAGCTGCGCAAGCTATCGCAAGCTGCAAGCTCAGAAAGGCAAGCTCGGCAAGCTTCGGCAAGCTTTGCAAGCTATGCAAGCTCTTGCAAGCTCTAGCAAGCTAGCAAGCTTTCTTGGCAAGCTAGCAAGCTCCTAGCAAGCTTCCGCAAGCTGGACCAGCAAGCTGCAAGCTTGCAAGCTTAACGCAAGCTGCAAGCTGCAAGCTGCAAGCTTCGCAAGCTCGCAAGCTGTCGCAAGCTTATGAATGGATGTCGCAAGCTGGGGCAAGCTGTCGCAAGCTCATTCGGCAAGCTCGCAAGCTGCAAGCTCTCCCCTCCGCGCAGCAAGCTGTTGCCGCAAGCTGGCAAGCTGCGTATGCAAGCTCATGGCAAGCTACCTAGCTGCTTTAGTGCAAGCTATAGCAAGCTGGAGCAAGCTTGGCAAGCTCGCAAGCTTAAATTTGCAAGCTGTTTGCAAGCTAGAGCAAGCTGATAGCCAGCAAGCTCCCTGCAAGCTGCAAGCTGTGCAAGCTGCAAGCTCTGCAAGCTAGAGAATCGCAAGCTTGCAAGCTGCAAGCTGCAAGCTTTCATTGGCACTTGACGCAAGCTTTGTGCAAGCTGCAAGCTGTCCTGAACACGCAAGCTGCAAGCTGCGCAAGCTGATTTGGCAAGCTGACGCAAGCTCGCAAGCTGCAAGCTGCAAGCTAACTTGGCAAGCTGCAAGCTGCAAGCTGCAAGCTGCAAGCTGAACGCAAGCTCAGGCCGCAAGCTCCAATAGCAAGCTGCAAGCTTCGCAAGCTGCGCAAGCTGCAAGCTGGACGCAAGCTCTGATCGCAAGCTTACTCTATCGCAAGCTTGCAAGCTGACTGCAAGCTATGCAAGCTTGTCACATTGCAAGCTGCAAGCTATACGCAAGCTAAAGCAAGCTCGTGCAAGCTCGCAAGCTCGTCGAGGCAAGCTTCTTGCAAGCTTCGCCGGCAAGCTGCAAGCTTGGCAAGCTGCAAGCTGTGCAAGCTGTTAGCAAGCTGCAAGCTAAAGAATTTCAAGGGCAAGCTAGAGCGCAAGCTAGCAAGCTGCAAGCTATGCGCAAGCTGCAAGCTTGCGCAAGCTGCAAGCTGGCAAGCTGTCGTCGCAAGCTGCAAGCTTGCAAGCTCACTACTGCAAGCTTGGCAGTCTACGCAAGCTTTGCAAGCTGGCAAGCTCGCAAGCTCATGCAAGCTCGGCAAGCTGGCAAGCTCGCAAGCTGAGGCAAGCTGCAAGCTGCAAGCTGCAAGCTGGAAACGCAAGCTCGCAAGCTGCAAGCTATGCAAGCTGTACACTGCAAGCTCAATGCAAGCTTAAGCGCAAGCTGCAAGCTGCAAGCTAAGTCGACTGCAAGCTGCAAGCTAGCTGGCGCTGCAAGCTGGCAAGCTGCAAGCTCGGCAAGCTTGCAAGCTCCCGACCCGCAAGCTGCAAGCTCTGCAAGCTGCAAGCTTGCAAGCTATGCAAGCTGGGCAAGCTGGCAAGCTGGGCAAGCTCGAAGCAAGCTTAGCAAGCTATAGGCAAGCTGCAAGCTCGCAAGCTCTGGCAAGCTCCATTGCAAGCTGCACGCAAGCTAGGCAAGCTACGCAAGCTGAGGAGCAAGCTCGCAAGCTCCCCGCAAGCTGGCAAGCTGCCCAGCAGCGTGTCTGATGCAAGCTCGGCAAGCTTCGCAAGCTGCAGTCGCTAGGAGTACGCAAGCTGCAAGCTGCAAGCTAGCAAGCTAGGTCCGAACGGCAAGCTCAATGCAAGCTCGGCAAGCTTGCAAGCTGTGGGCAAGCTAAGCAAGCTGGCAAGCTCGCAAGCTCCTAAGTGCAAGCTGCAAGCTGCAAGCTGCAAGCTGCGCAAGCTGGAGCAAGCTCGCAAGCTCTCCTCGCAAGCTGAGGCTGCAAGCTGCGATCGCAAGCTGCAAGCTCTGCAAGCTGGCAAGCTAATCCATGGCAAGCTATGCAAGCTCCGCAAGCTGAGCAAGCTAGGCAAGCTGACCGTCTGCAAGCTATGCAAGCTGGCAAGCTGCAAGCTGGCAAGCTGCAAGCTGCAAGCTGCAAGCTACCATCGGGCAAGCTACGCAAGCTCGTAAGAGCAAGCTGCAAGCTGAACGGGTAGCAAGCTGCAAGCTCGCAAGCTCCTGCAAGCTGCAAGCTGCAAGCTGCAAGCTTTAGTATGCAAGCTAAGCAAGCTGAGCAAGCTAACTCCGCAAGCTTCGCTGGCAAGCTTGCAAGCTCCTCGTTAGTAGCCGTAGCAAGCTCAAGCGAGGTGCAAGCTCCCGGCAAGCTATCAAACAAACCGGCAAGCTGCAAGCTAGATGCAAGCTTGTCCAACCTCGCAAGCTGCAAGCTGCAAGCTAGCAAGCTTTAAGCAAGCTGCAAGCTGTCATGCAAGCTATTGCAAGCTACTTGAAGTTTGCAAGCTGCAAGCTTAGCAAGCTGCGCAAGCTCTCATGCAAGCTACCTGGTCGTTGCAAGCTTGCAAGCTGGCAAGCTTCAAGCAAGCTCGCAAGCTTTATGCAAGCTGCAAGCTGCAAGCTCGCAAGCTGGCGCAAGCTTAGCAAGCTCCTTCCGCATAGCAAGCTTATGATGCAAGCTGCAAGCTGCAAGCTCGGCGGGCAAGCTCGACGGCAAGCTGCAAGCTAAACAGTACGCAAGCTGCAAGCTACCGCAAGCTCCAGCAAGCTTGCAAGCTGCGCGTAGCAAGCTGCAAGCTACTTGCGCAAGCTTTAGTGAACCCTACGCAAGCTAGCAAGCTGCAAGCTAAGGCAAGCTGGAACTCTGTTGCGCAAGCTCGCAAGCTGCAAGCTGCGTCGTGCAAGCTGGTGGCAAGCTCGCAAGCTACTTAGCAAGCTGCAAGCTCGTGGCAAGCTAGAGCAAGCTGCAAGCTGCAAGCTTGGCAAGCTACGGCAAGCTTCGGCAAGCTACCCGCAAGCTCGCAAGCTGCGCAAGCTAGCAAGCTCTTGAGCAAGCTTGCAAGCTAATGCTGCAAGCTGCAAGCTCTTAGCAAGCTGCAAGCTCTGCAAGCTGCAAGCTCGCCGGCAAGCTGCAGCAAGCTAGCAAGCTCAAGACAGACGCAAGCTCGCAAGCTTAGCAAGCTTGCAAGCTGCAAGCTAGCAAGCTGCAAGCTATTGCAAGCTCCAAGCCGCAAGCTGCAAGCTCGTATGCAAGCTGCAAGCTGCAAGCTTGGTTGTAGCCGCAAGCTCACGCGCTGCCCGGCAAGCTGCAAGCTGTTGCAAGCTTGGCAAGCTAGGCAAGCTTGCAAGCTACGCTCACAGCAAGCTTGCAAGCTTGCAAGCTAGCAAGCTGCAAGCTTGGCAAGCTGCAAGCTTAGGTGCAAGCTACGCAAGCTGCAAGCTCAGCAAGCTTACGTGCTCCAGCAAGCTTTGCAAGCTGATGCAAGCTACGCTTTCCGCAAGCTACAGGGCTGCGCCGGCAAGCTTAAGAGGCAAGCTAGATGCAAGCTGGCAAGCTCGGCAAGCTGTTTGCAAGCTATCCCCGGCAAGCTGCAAGCTGGGGGCAAGCTGCGGCAAGCTGCGCAAGCTTGCAAGCTGTGGGCAAGCTCCGCTTTTGCAAGCTCTGGCAAGCTCTGTAAGGCAAGCTGGCAAGCTCGCAAGCTCCCACGAGGCAAGCTGCAAGCTGCAAGCTAGCAAGCTGCGCAAGCTGCAAGCTCTACGGCAAGCTCCGATCGGCAAGCTTATGCAAGCTAGCTATCTCGCAAGCTTGCAAGCTGGAACGCTTTTGCAAGCTAGCAAGCTGCAAGCTCGCAAGCTGCAAGCTGTCATGCAAGCTGGCAAGCTCGGTCACTGCAAGCTACCGTAGCAAGCTTTCGCCGCAAGCTCGCAAGCTCAGCGTTACACAGCGCAAGCTAGCAAGCTCTACTTCCGCAAGCTTTCGCAAGCTGCAAGCTTTCTTATGCAAGCTGCAAGCTGCAAGCTGGCAAGCTAGTGCAAGCTGCAAGCTCGGCAAGCTCCGCAAGCTGACGGCAAGCTGCAAGCTGCAAGCTTAGCCGCAAGCTTCCAAAGCAGTAGCAAGCTGCAGTGCAAGCTAGGCAAGCTCAAGCAAGCTGAGTAGCAAGCTCTTAGCAAGCTGGCAAGCTCAGCAAGCTCGTACGCGAGCATCCGCAAGCTGCAAGCTTGCAAGCTCTACCGCAAGCTCGCAAGCTGCAAGCTAGCAAGCTGGCAAGCTGCGCAAGCTGCAAGCTGTGCAAGCTGCAAGCTTGCAAGCTCGCAAGCTGCAAGCTGCAAGCTGCAAGCTGCAAGCTGGGCAAGCTGCAAGCTCGAGCAAGCTGGCAAGCTGAGCAAGCTGGCAAGCTCGCAAGCTGCAAGCTAGTGAGGCAAGCTGCAAGCTGATCTAGCAAGCTTGCAAGCTAGGCAAGCTGGCAAGCT'
print(approximate_match(p, t, 0))

[9728, 1025, 2050, 8194, 8710, 9735, 2057, 7177, 525, 9742, 4114, 2073, 9243, 6172, 9758, 31, 2080, 3105, 4128, 5152, 5667, 6179, 38, 2599, 7201, 7208, 9258, 8243, 4149, 6199, 1592, 5178, 2108, 4156, 5693, 6206, 577, 2115, 5700, 6213, 2631, 4681, 5707, 5196, 6220, 6735, 4688, 9808, 4178, 2646, 6742, 1113, 8793, 5211, 1629, 6749, 1120, 8288, 611, 9828, 4198, 1127, 8295, 8809, 618, 108, 7794, 2165, 1654, 119, 3702, 6264, 2682, 6777, 1663, 8837, 1670, 6280, 137, 1161, 5259, 4748, 3213, 5116, 3727, 6289, 2706, 659, 8340, 4757, 7830, 7319, 1689, 7837, 7326, 4259, 2213, 7333, 5803, 7340, 9388, 6830, 8368, 5298, 180, 6837, 2743, 7863, 8375, 4794, 3262, 9409, 9416, 6860, 1230, 2767, 3793, 2778, 4315, 5852, 5342, 7393, 9441, 3811, 7910, 3818, 1772, 4333, 6380, 4847, 6892, 2289, 7406, 243, 8430, 7926, 3831, 1272, 765, 3326, 1279, 9477, 3852, 5389, 9484, 1295, 4879, 1809, 275, 6931, 793, 1311, 1824, 8480, 290, 2339, 6947, 8993, 297, 4906, 304, 4913, 8498, 9011, 9522, 2869, 826, 1338, 5435, 9021, 